# 04. Teams


## 1. Azure OpenAI 클라이언트 설정

AutoGen과 Azure OpenAI 서비스를 연동하기 위한 기본 설정입니다. 환경 변수에서 API 키와 엔드포인트 정보를 불러와 클라이언트를 초기화합니다.

In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

## 2. 단일 에이전트 팀 (Single Agent Team)

날씨 정보를 제공하는 간단한 AI 에이전트를 생성하고 실행하는 예제입니다. 
- `get_weather` 함수: 도시명을 입력받아 날씨 정보를 반환하는 도구
- `AssistantAgent`: 한국어로 대화하며 날씨 정보를 제공하는 에이전트
- `RoundRobinGroupChat`: 단일 에이전트로 구성된 팀 생성

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"{city}의 날씨는 섭씨 22도이고 맑습니다."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="작업이 완료되면 '완료'라고 응답하세요. 정중하고 친근한 한국어로 대화하세요.",
)

# Define a termination condition.
text_termination = TextMentionTermination("완료")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="서울의 날씨는 어떤가요?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()

messages=[TextMessage(id='c7f923ad-2b34-4e10-9ac3-dfcaf7b9bab0', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 56, 32, 619955, tzinfo=datetime.timezone.utc), content='서울의 날씨는 어떤가요?', type='TextMessage'), ToolCallRequestEvent(id='6fbfaf1d-d918-4cae-a19d-338423b9e598', source='assistant', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 56, 33, 554204, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_HnTjlGvfn41tPByRzODiLcTb', arguments='{"city":"서울"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='33f5c023-fb24-43aa-997a-08bbfb0336fb', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 56, 33, 555537, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='서울의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_HnTjlGvfn41tPByRzODiLcTb', is_error=False)], t

## 3. 스트리밍 모드로 팀 실행

동일한 단일 에이전트 팀을 스트리밍 모드로 실행하는 방법입니다. 실시간으로 메시지를 받아보고 작업 완료 상태를 확인할 수 있습니다.
- `run_stream()`: 실시간 메시지 스트리밍
- `TaskResult`: 작업 완료 시 중단 이유 출력

In [3]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="부산의 날씨는 어떤가요?"):
        if isinstance(message, TaskResult):
            print("중단 이유:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()

id='0d5a2d19-ce81-4b1c-850d-718220917196' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 11, 1, 9, 57, 10, 735182, tzinfo=datetime.timezone.utc) content='부산의 날씨는 어떤가요?' type='TextMessage'
id='bdae2121-d29b-4bab-a554-846d3fa1a393' source='assistant' models_usage=RequestUsage(prompt_tokens=183, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 11, 1, 9, 57, 11, 459306, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_1tt0AazlPHfw8SFerDI43YJg', arguments='{"city":"부산"}', name='get_weather')] type='ToolCallRequestEvent'
id='b3be1628-8fe5-404a-8077-8ce40c32b85f' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 11, 1, 9, 57, 11, 460760, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='부산의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_1tt0AazlPHfw8SFerDI43YJg', is_error=False)] type='ToolCallExecutionEvent'
id='564b95c8-cbee-41db-9db3-41fea0d17c5b' source='assistan

## 4. 콘솔 UI를 활용한 대화형 실행

AutoGen의 콘솔 UI를 사용하여 더 직관적으로 에이전트와의 대화를 확인할 수 있습니다.
- `Console()`: 대화형 콘솔 인터페이스 제공
- `reset()`: 팀 상태 초기화로 새로운 대화 시작
- 영어 예제로 다국어 지원 확인

In [4]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_6at2qmMFVRf9RgqfJB7xzThh', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='Seattle의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_6at2qmMFVRf9RgqfJB7xzThh', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
Seattle의 날씨는 섭씨 22도이고 맑습니다.
---------- TextMessage (assistant) ----------
시애틀의 날씨는 현재 섭씨 22도이며, 맑은 상태입니다. 도움이 필요하시면 언제든지 말씀해 주세요!
---------- TextMessage (assistant) ----------
완료


TaskResult(messages=[TextMessage(id='50b1b359-0880-4b30-a84d-aad05f61da7d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 57, 23, 667841, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(id='110a743c-32c4-4ce6-bb28-a5b1e34adb11', source='assistant', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 57, 24, 252446, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_6at2qmMFVRf9RgqfJB7xzThh', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='a865eb14-d4fa-450e-83ed-61fe8a078101', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 57, 24, 253433, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Seattle의 날씨는 섭씨 22도이고 맑습니다.', name='get_weather', call_id='call_6at2qmM

## 5. 반성 팀 (Reflection Team) - 다중 에이전트 협업

두 개의 에이전트가 협업하여 작업을 수행하는 반성 패턴입니다.
- **Primary Agent**: 주요 작업을 수행하는 AI 어시스턴트
- **Critic Agent**: 결과물을 검토하고 건설적인 피드백 제공
- **협업 프로세스**: Primary가 작업 → Critic이 검토 → 피드백 반영 → 승인 시 완료
- **종료 조건**: "승인합니다" 키워드 또는 최대 15개 메시지 제한

In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="당신은 도움이 되는 AI 어시스턴트입니다. 정중하고 친근한 한국어로 대화하며, 정확하고 유용한 정보를 제공하세요.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="건설적인 피드백을 제공하세요. 피드백이 반영되었을 때 '승인합니다'라고 응답하세요. 정중하고 예의바른 한국어로 소통하세요.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("승인합니다")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(15)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

### 5-1. 반성 팀 실행 - 가을 시 창작

반성 팀을 실행하여 가을에 대한 시를 창작하는 예제입니다. Primary 에이전트가 시를 작성하면 Critic 에이전트가 검토하고 피드백을 제공합니다.

In [7]:

# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="가을에 대한 짧은 시를 써보세요.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
가을에 대한 짧은 시를 써보세요.
---------- TextMessage (primary) ----------
가을바람 속에 나뭇잎이 춤추고,  
황금빛 들판이 햇살을 머금네.  
서늘한 공기에 마음을 열고,  
차라리 이 순간, 영원하길 바라네.  

하늘은 높고 구름은 가벼워,  
자연의 속삭임이 귀 기울이네.  
가을의 정취에 젖어드는 나,  
온 세상이 빛나는, 찬란한 소풍길.  
---------- TextMessage (critic) ----------
아주 아름다운 시입니다. 가을의 정취와 그 순간의 소중함을 잘 표현하셨네요. 특히 자연의 모습과 감정을 잘 연결한 점이 인상적입니다. 조금 더 구체적인 이미지나 감정을 추가한다면 더 깊이 있는 감상을 불러일으킬 수 있을 것 같아요. 예를 들면, 나뭇잎의 색깔이나 바람의 속삭임 등을 조금 더 자세히 묘사하면 좋을 것 같습니다. 

전체적으로 훌륭한 시이므로, 저의 작은 제안이 도움이 되기를 바랍니다. 

반영하신다면 '승인합니다'라고 말씀드리겠습니다.


TaskResult(messages=[TextMessage(id='d9928a94-31d4-4ef4-95be-1d9e86b25675', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 17, 57900, tzinfo=datetime.timezone.utc), content='가을에 대한 짧은 시를 써보세요.', type='TextMessage'), TextMessage(id='c24ea876-fdbd-4b9d-bbef-dcce12d140cd', source='primary', models_usage=RequestUsage(prompt_tokens=61, completion_tokens=117), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 18, 400585, tzinfo=datetime.timezone.utc), content='가을바람 속에 나뭇잎이 춤추고,  \n황금빛 들판이 햇살을 머금네.  \n서늘한 공기에 마음을 열고,  \n차라리 이 순간, 영원하길 바라네.  \n\n하늘은 높고 구름은 가벼워,  \n자연의 속삭임이 귀 기울이네.  \n가을의 정취에 젖어드는 나,  \n온 세상이 빛나는, 찬란한 소풍길.  ', type='TextMessage'), TextMessage(id='8360905a-a686-4256-b31b-fb11256a4a65', source='critic', models_usage=RequestUsage(prompt_tokens=192, completion_tokens=148), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 20, 158851, tzinfo=datetime.timezone.utc), content="아주 아름다운 시입니다. 가을의 정취와 그 순간의 소중함을 잘 표현

### 5-2. 반성 팀 실행 - 당나라 시풍 창작

더 복잡한 창작 작업으로 중국 당나라 시풍의 시를 창작하는 예제입니다. 특정 문학적 스타일을 요구하는 고급 작업에서 두 에이전트가 어떻게 협업하는지 확인할 수 있습니다.

In [8]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
가을 하늘 높고 구름은 가벼워,  
단풍잎 붉게 물들어 버드나무 아래.  
산빛은 잔잔하고 계곡은 고요해,  
풀 내음 가득한 이 길을 따라 걸어가네.

석양이 지면 황금빛이 감도니,  
아이들의 웃음소리 멀리 퍼지네.  
간혹 스치는 가을바람,  
그리운 옛 노래를 속삭이는 듯.  

차 한 잔에 담긴 계절의 맛,  
조용한 시간 속에 마음을 비우네.  
세월의 흐름이여, 여유를 주고,  
가을의 정취에 느림을 더하소서.  
---------- TextMessage (critic) ----------
매우 잘 쓰신 시입니다! 당나라 시풍의 감각을 잘 살리면서도 가을의 풍경과 감성을 아름답게 표현하셨습니다. 특히 자연의 아름다움과 그 속에서 느끼는 감정을 매우 잘 어우러지게 하신 것이 인상적입니다.

다만, 혹시 고전적인 표현을 조금 더 사용하거나 특정한 형식을 반영하면 더욱 당나라 시풍에 가까운 느낌을 줄 수 있을 것 같습니다. 예를 들어, 대칭적인 표현이나 운율을 조금 더 강조하면 좋을 것 같습니다.

전반적으로 훌륭한 작품이니, 저의 제안이 도움이 되길 바랍니다. 

반영하신다면 '승인합니다'라고 말씀드리겠습니다.


TaskResult(messages=[TextMessage(id='789fb193-f91d-4489-bb1d-29735fbfbe77', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 24, 58526, tzinfo=datetime.timezone.utc), content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(id='6a1f2e08-ee72-4f39-9b1e-fc74c4588e60', source='primary', models_usage=RequestUsage(prompt_tokens=353, completion_tokens=171), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 25, 749248, tzinfo=datetime.timezone.utc), content='가을 하늘 높고 구름은 가벼워,  \n단풍잎 붉게 물들어 버드나무 아래.  \n산빛은 잔잔하고 계곡은 고요해,  \n풀 내음 가득한 이 길을 따라 걸어가네.\n\n석양이 지면 황금빛이 감도니,  \n아이들의 웃음소리 멀리 퍼지네.  \n간혹 스치는 가을바람,  \n그리운 옛 노래를 속삭이는 듯.  \n\n차 한 잔에 담긴 계절의 맛,  \n조용한 시간 속에 마음을 비우네.  \n세월의 흐름이여, 여유를 주고,  \n가을의 정취에 느림을 더하소서.  ', type='TextMessage'), TextMessage(id='9eb03fba-ccd2-4465-a449-f75e2dc6d6ed', source='critic', models_usage=RequestUsage(prompt_tokens=538, completion_tokens=165), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 5

## 6. 핸드오프 팀 (Handoff Team) - 사용자 전달 패턴

답변을 모르는 질문이나 복잡한 요청을 사용자에게 전달하는 패턴입니다.
- **Lazy Agent**: 답변을 모를 때 사용자에게 전달하는 에이전트
- **Handoff 메커니즘**: 특정 조건에서 사용자에게 제어권 전달
- **실용적 활용**: 에이전트의 한계를 인정하고 인간의 도움을 요청하는 겸손한 AI 패턴

In [10]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="사용자에게 전달합니다.")],
    system_message="답을 모를 때는 항상 사용자에게 전달하세요. 작업이 완료되면 '완료'라고 응답하세요. 정중하고 친근한 한국어로 대화하세요.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("완료")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

### 6-1. 핸드오프 팀 실행 예제

두 가지 다른 유형의 질문으로 핸드오프 팀의 동작을 테스트합니다:
1. **"서울 날씨 어때?"** - 구체적인 날씨 질문 (통계 정보 포함)
2. **"날씨 맑은데?"** - 모호한 질문으로 사용자 전달 가능성 테스트

각 질문에 대해 에이전트가 어떻게 반응하고 언제 사용자에게 전달하는지 관찰할 수 있습니다.

In [11]:
from autogen_agentchat.ui import Console

await Console(lazy_agent_team.run_stream(task="서울 날씨 어때?"),  output_stats=True)

# Handoff to the user before answering the question

await Console(lazy_agent_team.run_stream(task="날씨 맑은데?"))

---------- TextMessage (user) ----------
서울 날씨 어때?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_HQilTYQtV5pBBMfTa0segghJ', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 83, Completion tokens: 12]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='사용자에게 전달합니다.', name='transfer_to_user', call_id='call_HQilTYQtV5pBBMfTa0segghJ', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
사용자에게 전달합니다.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 83
Total completion tokens: 12
Duration: 0.66 seconds
---------- TextMessage (user) ----------
날씨 맑은데?
---------- TextMessage (lazy_assistant) ----------
아, 날씨가 맑다니 좋네요! 그런 날은 밖에 나가서 산책하기에도 딱 좋은 날씨죠. 계획하고 계신 활동이 있으신가요?
---------- TextMessage (lazy_assistant) ----------
무엇을 도와드릴까요?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id=

TaskResult(messages=[TextMessage(id='212421df-66c9-4ba5-9087-382b3e493b34', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 41, 206791, tzinfo=datetime.timezone.utc), content='날씨 맑은데?', type='TextMessage'), TextMessage(id='5ef76bf0-ebcb-4fd6-b6be-5ce3e92f4843', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=44), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 42, 129048, tzinfo=datetime.timezone.utc), content='아, 날씨가 맑다니 좋네요! 그런 날은 밖에 나가서 산책하기에도 딱 좋은 날씨죠. 계획하고 계신 활동이 있으신가요?', type='TextMessage'), TextMessage(id='cbf45db1-a5cc-4097-9b92-e9f382eda21b', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=11), metadata={}, created_at=datetime.datetime(2025, 11, 1, 9, 58, 42, 718355, tzinfo=datetime.timezone.utc), content='무엇을 도와드릴까요?', type='TextMessage'), ToolCallRequestEvent(id='ff8b9f83-06e4-4ffb-beee-20158feae81b', source='lazy_assistant', models_us

---

# 종합 실습 과제: 실시간 API 연동 스마트 여행 계획 AI 팀 만들기

지금까지 학습한 내용을 바탕으로 **실제 API를 활용한** 여행 계획 다중 에이전트 시스템을 만들어보겠습니다.

## 📋 실습 목표 & 실제 API 연동
1. **여행지 전문가**: 🌐 **Google Places API**로 실시간 명소/맛집/숙박 추천
2. **날씨 컨설턴트**: 🌤️ **OpenWeatherMap API**로 실시간 날씨 예보 및 여행 조언
3. **예산 관리자**: 💰 여행 비용 계산 및 예산 조언  
4. **여행 컨시어지**: 🛎️ 복잡한 요청 시 전문가에게 핸드오프

## 🆕 API 활용 특징
- **Google Places API**: 실제 관광지, 맛집, 숙박시설 정보 + 평점
- **OpenWeatherMap API**: 실시간 날씨 예보 + 날씨 기반 여행 추천
- **Fallback 시스템**: API 키 없이도 기본 데이터로 학습 가능

이 실습을 통해 **실제 API 연동**, **단일 에이전트**, **반성 팀**, **핸드오프 패턴**을 모두 활용하는 종합적인 시스템을 구축해보세요!

In [21]:
# ============================================================================
# 🎯 실습 과제: 스마트 여행 계획 AI 팀 구현하기
# ============================================================================

# 필요한 라이브러리 import
import aiohttp
import json
import random
from datetime import datetime, timedelta

# 1. 여행지 추천 도구 함수들 정의 (Google Places API 활용)
async def get_popular_attractions(city: str, category: str = "관광지") -> str:
    """Google Places API를 사용해서 실제 명소를 검색합니다."""
    google_api_key = os.getenv("GOOGLE_PLACES_API_KEY")
    
    if not google_api_key:
        # API 키가 없으면 기본 데이터베이스 사용
        attractions_db = {
            "서울": {
                "관광지": ["경복궁", "남산타워", "명동", "홍대", "동대문디자인플라자"],
                "맛집": ["광장시장", "명동교자", "신촌맛집거리", "이태원"],
                "숙박": ["롯데호텔", "신라호텔", "게스트하우스", "에어비앤비"]
            },
            "부산": {
                "관광지": ["해운대해수욕장", "광안대교", "감천문화마을", "태종대", "광안리해수욕장"],
                "맛집": ["자갈치시장", "해운대맛집거리", "서면맛집"],
                "숙박": ["해운대호텔", "부산역근처숙소", "게스트하우스"]
            },
            "제주": {
                "관광지": ["성산일출봉", "한라산", "우도", "천지연폭포", "협재해수욕장"],
                "맛집": ["흑돼지거리", "동문시장", "제주향토음식"],
                "숙박": ["제주신화월드", "펜션", "리조트"]
            },
            "경주": {
                "관광지": ["불국사", "석굴암", "첨성대", "안압지", "대릉원"],
                "맛집": ["경주전통음식", "황남빵", "보리밥집"],
                "숙박": ["경주호텔", "한옥펜션", "게스트하우스"]
            }
        }
        
        city_data = attractions_db.get(city, {"관광지": ["정보를 찾을 수 없습니다"]})
        results = city_data.get(category, ["해당 카테고리 정보가 없습니다"])
        
        return f"{city}의 {category} 추천:\n" + "\n".join([f"- {item}" for item in results])
    
    try:
        # Google Places API를 사용한 실제 검색
        place_type_mapping = {
            "관광지": "tourist_attraction",
            "맛집": "restaurant", 
            "숙박": "lodging",
            "쇼핑": "shopping_mall",
            "카페": "cafe"
        }
        
        place_type = place_type_mapping.get(category, "tourist_attraction")
        
        async with aiohttp.ClientSession() as session:
            # Text Search API 사용
            search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
            params = {
                "query": f"{city} {category}",
                "type": place_type,
                "key": google_api_key,
                "language": "ko"
            }
            
            async with session.get(search_url, params=params) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 {category} 정보를 가져올 수 없습니다."
                
                data = await response.json()
                
                if not data.get("results"):
                    return f"{city}에서 {category} 정보를 찾을 수 없습니다."
                
                # 상위 10개 결과 추출
                places = []
                for place in data["results"][:10]:
                    name = place.get("name", "이름 없음")
                    rating = place.get("rating", 0)
                    
                    place_info = f"{name}"
                    if rating > 0:
                        place_info += f" (평점: {rating})"
                    places.append(place_info)
                
                result = f"{city}의 {category} 추천 (Google Places API):\n"
                result += "\n".join([f"- {place}" for place in places])
                
                return result
                
    except Exception as e:
        return f"{category} 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

async def calculate_travel_budget(city: str, days: int, style: str = "보통") -> str:
    """여행 예산을 계산합니다."""
    base_costs = {
        "서울": {"숙박": 80000, "식사": 50000, "교통": 30000, "관광": 40000},
        "부산": {"숙박": 60000, "식사": 40000, "교통": 25000, "관광": 35000},
        "제주": {"숙박": 100000, "식사": 60000, "교통": 40000, "관광": 50000},
        "경주": {"숙박": 50000, "식사": 35000, "교통": 20000, "관광": 30000},
    }
    
    if city not in base_costs:
        return f"{city}의 예산 정보를 찾을 수 없습니다."
    
    multiplier = {"절약": 0.7, "보통": 1.0, "럭셔리": 1.5}.get(style, 1.0)
    daily_cost = sum(base_costs[city].values()) * multiplier
    total_cost = daily_cost * days
    
    return f"{city} {days}일 여행 ({style} 스타일): 총 {total_cost:,.0f}원 (일일 {daily_cost:,.0f}원)"

async def get_weather_recommendation(city: str, days: int = 3) -> str:
    """OpenWeatherMap API를 사용해서 날씨 예보와 여행 추천사항을 제공합니다."""
    api_key = os.getenv("OPENWEATHER_API_KEY")
    
    if not api_key:
        # API 키가 없으면 모의 데이터 제공
        weather_conditions = ["맑음", "흐림", "비", "눈", "안개"]
        temperatures = list(range(15, 30))
        
        forecast = {"city": city, "forecast": []}
        
        for i in range(days):
            date = datetime.now() + timedelta(days=i)
            day_weather = {
                "date": date.strftime("%Y-%m-%d"),
                "condition": random.choice(weather_conditions),
                "temperature": f"{random.choice(temperatures)}°C",
                "humidity": f"{random.randint(40, 80)}%"
            }
            forecast["forecast"].append(day_weather)
        
        # 계절별 추천사항 추가
        current_month = datetime.now().month
        if current_month in [3, 4, 5]:
            season = "봄"
        elif current_month in [6, 7, 8]:
            season = "여름"
        elif current_month in [9, 10, 11]:
            season = "가을"
        else:
            season = "겨울"
            
        recommendations = {
            ("서울", "봄"): "벚꽃이 아름다운 계절입니다. 여의도 한강공원 추천!",
            ("서울", "여름"): "더위를 피해 실내 관광지나 한강 야경을 즐기세요.",
            ("서울", "가을"): "단풍이 아름다운 남산과 북촌한옥마을을 추천합니다.",
            ("서울", "겨울"): "스케이트장과 크리스마스 마켓을 즐기세요.",
            ("부산", "여름"): "해수욕장과 바다 활동을 즐기기 최적의 시기입니다!",
            ("제주", "봄"): "유채꽃과 벚꽃이 아름다운 계절입니다.",
        }
        
        season_tip = recommendations.get((city, season), f"{city}의 {season} 여행을 즐기세요!")
        
        result = json.dumps(forecast, ensure_ascii=False, indent=2)
        result += f"\n\n🌟 계절별 추천: {season_tip}"
        
        return result
    
    try:
        # 실제 OpenWeatherMap API 호출
        async with aiohttp.ClientSession() as session:
            # 1. 도시 좌표 얻기
            geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
            async with session.get(geo_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                geo_data = await response.json()
                if not geo_data:
                    return f"죄송합니다. {city}의 위치 정보를 찾을 수 없습니다."
                
                lat, lon = geo_data[0]['lat'], geo_data[0]['lon']
            
            # 2. 날씨 예보 가져오기
            weather_url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric&lang=kr"
            async with session.get(weather_url) as response:
                if response.status != 200:
                    return f"죄송합니다. {city}의 날씨 정보를 가져올 수 없습니다."
                
                weather_data = await response.json()
                
                # 일별 예보 정리
                daily_forecasts = {}
                for item in weather_data['list'][:days*8]:  # 3시간 간격이므로 하루 8개
                    date = item['dt_txt'][:10]  # YYYY-MM-DD
                    if date not in daily_forecasts:
                        daily_forecasts[date] = {
                            "date": date,
                            "condition": item['weather'][0]['description'],
                            "temperature": f"{int(item['main']['temp'])}°C",
                            "humidity": f"{item['main']['humidity']}%"
                        }
                
                forecast = {
                    "city": city,
                    "forecast": list(daily_forecasts.values())[:days]
                }
                
                # 날씨 기반 여행 추천 추가
                avg_temp = sum([int(d['temperature'][:-2]) for d in forecast['forecast']]) / len(forecast['forecast'])
                
                if avg_temp > 25:
                    weather_tip = "더운 날씨예요! 시원한 실내 관광지나 수영장, 해변 활동을 추천합니다."
                elif avg_temp < 10:
                    weather_tip = "추운 날씨예요! 따뜻한 실내 명소나 온천, 찜질방을 추천합니다."
                else:
                    weather_tip = "야외 활동하기 좋은 날씨입니다! 산책로나 공원 관광을 즐기세요."
                
                result = json.dumps(forecast, ensure_ascii=False, indent=2)
                result += f"\n\n🌟 날씨 기반 추천: {weather_tip}"
                
                return result
                
    except Exception as e:
        return f"날씨 정보를 가져오는 중 오류가 발생했습니다: {str(e)}"

In [22]:
# 2. 여행지 전문가 에이전트 (단일 에이전트 팀)
travel_expert = AssistantAgent(
    "travel_expert",
    model_client=azure_openai_chat_completion_client,
    tools=[get_popular_attractions, get_weather_recommendation],
    system_message="""당신은 한국 여행 전문가입니다. 
    - Google Places API와 OpenWeatherMap API를 활용해 실시간 정보를 제공하세요
    - 고객의 여행 목적지에 대한 실제 관광명소, 맛집, 숙박시설을 추천하세요
    - 현재 날씨 상황을 고려한 여행 팁과 주의사항을 제공하세요
    - get_popular_attractions 함수는 city와 category(관광지, 맛집, 숙박 등) 파라미터를 받습니다
    - get_weather_recommendation 함수는 city와 days 파라미터를 받아 날씨 예보와 추천사항을 제공합니다
    - 정중하고 친근한 한국어로 대화하며, 작업 완료 시 '추천 완료'라고 말하세요""",
)

# 여행지 전문가 팀 생성
expert_termination = TextMentionTermination("추천 완료")
travel_expert_team = RoundRobinGroupChat([travel_expert], termination_condition=expert_termination)

In [23]:
# 3. 예산 관리 팀 (반성 패턴 - Primary + Critic)

# 예산 계산 에이전트
budget_planner = AssistantAgent(
    "budget_planner", 
    model_client=azure_openai_chat_completion_client,
    tools=[calculate_travel_budget],
    system_message="""당신은 여행 예산 전문가입니다.
    - 고객의 여행 계획에 맞는 현실적인 예산을 계산하세요
    - 절약 팁과 예산 최적화 방안을 제안하세요
    - 정확한 숫자와 근거를 제시하며 정중한 한국어로 소통하세요""",
)

# 예산 검토 에이전트  
budget_critic = AssistantAgent(
    "budget_critic",
    model_client=azure_openai_chat_completion_client,
    system_message="""당신은 여행 예산 검토 전문가입니다.
    - 제시된 예산이 현실적이고 합리적인지 검토하세요
    - 누락된 비용이나 과도한 예산이 있는지 점검하세요
    - 검토 완료 시 '예산 승인합니다'라고 응답하세요
    - 건설적인 피드백을 정중한 한국어로 제공하세요""",
)

# 예산 관리 팀 생성 (반성 패턴)
budget_termination = TextMentionTermination("예산 승인합니다") | MaxMessageTermination(10)
budget_management_team = RoundRobinGroupChat([budget_planner, budget_critic], termination_condition=budget_termination)

In [24]:
# 4. 여행 컨시어지 (핸드오프 패턴)

travel_concierge = AssistantAgent(
    "travel_concierge",
    model_client=azure_openai_chat_completion_client,
    handoffs=[Handoff(target="travel_advisor", message="전문 상담사에게 연결합니다")],
    system_message="""당신은 여행 컨시어지입니다.
    - 간단한 질문은 직접 답변하고, 복잡하거나 전문적인 상담이 필요한 경우 전문가에게 연결하세요
    - 항공편, 비자, 보험 등 전문 지식이 필요한 문의는 전문가에게 전달하세요  
    - 작업 완료 시 '안내 완료'라고 응답하세요
    - 항상 정중하고 도움이 되는 한국어로 소통하세요""",
)

# 컨시어지 팀 생성
concierge_termination = HandoffTermination(target="travel_advisor") | TextMentionTermination("안내 완료")
concierge_team = RoundRobinGroupChat([travel_concierge], termination_condition=concierge_termination)

## 🧪 실습 테스트: 실시간 API 연동 기능 확인

이제 만든 **실시간 API 연동 AI 팀들**을 실제로 테스트해보겠습니다. 각 팀이 어떻게 실제 데이터로 작동하는지 관찰해보세요!

### 📡 API 연동 상태
- **Google Places API**: 실제 관광지/맛집 검색 (API 키 있을 경우)
- **OpenWeatherMap API**: 실시간 날씨 예보 (API 키 있을 경우)  
- **Fallback**: API 키가 없어도 기본 데이터로 동작

In [25]:
# 테스트 1: 여행지 전문가 팀 - Google Places API + OpenWeatherMap API 활용
print("=== 🗺️ 여행지 전문가 팀 테스트 (실시간 API 연동) ===")
await Console(travel_expert_team.run_stream(task="부산 여행 계획 중인데, 실제 관광명소와 맛집 추천하고 현재 날씨도 확인해주세요!"))

=== 🗺️ 여행지 전문가 팀 테스트 ===
---------- TextMessage (user) ----------
부산 여름 여행 계획 중인데, 추천 명소와 계절별 팁 알려주세요!
---------- ToolCallRequestEvent (travel_expert) ----------
[FunctionCall(id='call_a3lnfYYGqYSR2fyhHIqxFVzY', arguments='{"city": "부산", "category": "관광지"}', name='get_popular_attractions'), FunctionCall(id='call_5znfXGBQM9bgdCoXbAJR43hM', arguments='{"city": "부산", "category": "맛집"}', name='get_popular_attractions'), FunctionCall(id='call_tN3G2wBnPGXanKozDsMSyRjZ', arguments='{"city": "부산", "category": "숙박"}', name='get_popular_attractions')]
---------- ToolCallExecutionEvent (travel_expert) ----------
[FunctionExecutionResult(content='부산의 관광지 추천 (Google Places API):\n- 감천문화마을 (평점: 4.4)\n- 용두산공원 (평점: 4.3)\n- BIFF광장 (평점: 4.1)\n- 오륙도 스카이워크 (평점: 4.2)\n- 청사포 (평점: 4.4)\n- 해동용궁사 (평점: 4.4)\n- 송도용궁구름다리 (평점: 4.3)\n- 흰여울문화마을 (평점: 4.4)\n- 40계단 문화관광테마거리 (평점: 3.9)\n- 부산시민공원 (평점: 4.6)', name='get_popular_attractions', call_id='call_a3lnfYYGqYSR2fyhHIqxFVzY', is_error=False), FunctionExecutionResult(c

TaskResult(messages=[TextMessage(id='cde13c5f-e6b8-42da-b5da-4e249df560e6', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 36, 526501, tzinfo=datetime.timezone.utc), content='부산 여름 여행 계획 중인데, 추천 명소와 계절별 팁 알려주세요!', type='TextMessage'), ToolCallRequestEvent(id='a0b01964-3f8d-4908-997b-0b32c2e74009', source='travel_expert', models_usage=RequestUsage(prompt_tokens=312, completion_tokens=87), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 37, 802941, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_a3lnfYYGqYSR2fyhHIqxFVzY', arguments='{"city": "부산", "category": "관광지"}', name='get_popular_attractions'), FunctionCall(id='call_5znfXGBQM9bgdCoXbAJR43hM', arguments='{"city": "부산", "category": "맛집"}', name='get_popular_attractions'), FunctionCall(id='call_tN3G2wBnPGXanKozDsMSyRjZ', arguments='{"city": "부산", "category": "숙박"}', name='get_popular_attractions')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='f

In [26]:
# 테스트 2: 예산 관리 팀 - 반성 패턴 (협업)
print("\n=== 💰 예산 관리 팀 테스트 (Primary + Critic 협업) ===")
await Console(budget_management_team.run_stream(task="서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!"))


=== 💰 예산 관리 팀 테스트 (Primary + Critic 협업) ===
---------- TextMessage (user) ----------
서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!
---------- ToolCallRequestEvent (budget_planner) ----------
[FunctionCall(id='call_vzMP1IL1E0L1SltdhzxZoRLc', arguments='{"city":"서울","days":4,"style":"럭셔리"}', name='calculate_travel_budget')]
---------- ToolCallExecutionEvent (budget_planner) ----------
[FunctionExecutionResult(content='서울 4일 여행 (럭셔리 스타일): 총 1,200,000원 (일일 300,000원)', name='calculate_travel_budget', call_id='call_vzMP1IL1E0L1SltdhzxZoRLc', is_error=False)]
---------- ToolCallSummaryMessage (budget_planner) ----------
서울 4일 여행 (럭셔리 스타일): 총 1,200,000원 (일일 300,000원)
---------- TextMessage (budget_critic) ----------
제시하신 서울 3박 4일 가족 여행 예산(총 1,200,000원, 일일 300,000원)에 대해 검토해보았습니다. 

1. **숙박 비용**: 럭셔리 스타일로 원하신다면 고급 호텔에서 숙박하는 것이 좋습니다. 서울의 5성급 호텔은 1박에 20만 원에서 50만 원 이상입니다. 따라서 3박에 60만 원에서 150만 원 이상의 예산이 필요할 수 있습니다.

2. **식사 비용**: 럭셔리 식사를 고려하면, 한 끼에 2만 원에서 5만 원 정도로 잡고, 하루에 3끼 식사를 하시면 하루에 약 6만 원에서 15만 

TaskResult(messages=[TextMessage(id='b93fdbd0-f47a-4de5-ae29-8737a84664fc', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 48, 864513, tzinfo=datetime.timezone.utc), content='서울 3박 4일 가족 여행 예산을 계산해주세요. 럭셔리 스타일로 가고 싶어요!', type='TextMessage'), ToolCallRequestEvent(id='f2a5335a-3b8f-40df-888d-65aae6919d82', source='budget_planner', models_usage=RequestUsage(prompt_tokens=171, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 49, 699000, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_vzMP1IL1E0L1SltdhzxZoRLc', arguments='{"city":"서울","days":4,"style":"럭셔리"}', name='calculate_travel_budget')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='5bec4285-a411-44af-949c-9f1a07588ff3', source='budget_planner', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 49, 700316, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='서울 4일 여행 (럭셔리 스

In [27]:
# 테스트 3: 컨시어지 팀 - 핸드오프 패턴 
print("\n=== 🛎️ 여행 컨시어지 팀 테스트 (핸드오프 패턴) ===")
print("간단한 질문:")
await Console(concierge_team.run_stream(task="여행 가방 짐싸기 팁 알려주세요"))


=== 🛎️ 여행 컨시어지 팀 테스트 (핸드오프 패턴) ===
간단한 질문:
---------- TextMessage (user) ----------
여행 가방 짐싸기 팁 알려주세요


---------- TextMessage (travel_concierge) ----------
여행 가방 짐 싸기에 도움이 될 몇 가지 팁을 알려드릴게요:

1. **짧게 기획하기**: 여행 일수를 고려하여 필요한 의류와 물건들을 최소한으로 준비하세요. 여행의 목적과 날씨에 따라 필요한 아이템을 미리 정리해 봅니다.

2. **롤링 기술 사용**: 옷을 접는 대신 롤링하여 싸면 공간을 절약할 수 있고, 주름이 덜 생깁니다.

3. **파우치 활용**: 작은 물건들은 파우치에 넣어 정리하면 가방 안에서 이동할 때 분실되지 않고, 찾기에도 쉽습니다.

4. **필수 아이템 우선 순위**: 가장 필요한 물품을 맨 위에 배치하세요. 공항 수속이나 체크인 시 쉽게 꺼낼 수 있도록 배치하는 것이 좋습니다.

5. **신발에 짐 넣기**: 신발 속에 양말이나 작은 물건을 넣어 공간을 효율적으로 사용할 수 있습니다.

6. **전자기기 및 충전기**: 전자기기와 충전기는 따로 관리하고, 수하물에 넣지 않는 것이 좋습니다. 꼭 필요할 경우 수납용 파우치에 함께 포장하세요.

7. **넉넉한 공간 확보**: 여행 중 기념품을 구매할 계획이 있다면, 가방에 여유 공간을 남겨두세요.

이런 팁들을 참고하여 효과적으로 짐을 싸보세요! 도움이 더 필요하시면 말씀해 주세요.
---------- TextMessage (travel_concierge) ----------
안내 완료


TaskResult(messages=[TextMessage(id='3714ac93-b9d7-4fb2-a521-b4af89e63239', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 20, 58, 517077, tzinfo=datetime.timezone.utc), content='여행 가방 짐싸기 팁 알려주세요', type='TextMessage'), TextMessage(id='3fe53a60-8a61-4181-9962-86584a107966', source='travel_concierge', models_usage=RequestUsage(prompt_tokens=144, completion_tokens=346), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 21, 4, 178236, tzinfo=datetime.timezone.utc), content='여행 가방 짐 싸기에 도움이 될 몇 가지 팁을 알려드릴게요:\n\n1. **짧게 기획하기**: 여행 일수를 고려하여 필요한 의류와 물건들을 최소한으로 준비하세요. 여행의 목적과 날씨에 따라 필요한 아이템을 미리 정리해 봅니다.\n\n2. **롤링 기술 사용**: 옷을 접는 대신 롤링하여 싸면 공간을 절약할 수 있고, 주름이 덜 생깁니다.\n\n3. **파우치 활용**: 작은 물건들은 파우치에 넣어 정리하면 가방 안에서 이동할 때 분실되지 않고, 찾기에도 쉽습니다.\n\n4. **필수 아이템 우선 순위**: 가장 필요한 물품을 맨 위에 배치하세요. 공항 수속이나 체크인 시 쉽게 꺼낼 수 있도록 배치하는 것이 좋습니다.\n\n5. **신발에 짐 넣기**: 신발 속에 양말이나 작은 물건을 넣어 공간을 효율적으로 사용할 수 있습니다.\n\n6. **전자기기 및 충전기**: 전자기기와 충전기는 따로 관리하고, 수하물에 넣지 않는 것이 좋

In [28]:
# 복잡한 질문으로 핸드오프 테스트
print("\n복잡한 질문 (핸드오프 예상):")
await Console(concierge_team.run_stream(task="유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요"))


복잡한 질문 (핸드오프 예상):
---------- TextMessage (user) ----------
유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요
---------- ToolCallRequestEvent (travel_concierge) ----------
[FunctionCall(id='call_pfsYzFfKVXUhzAzlYohQJSt0', arguments='{}', name='transfer_to_travel_advisor')]
---------- ToolCallExecutionEvent (travel_concierge) ----------
[FunctionExecutionResult(content='전문 상담사에게 연결합니다', name='transfer_to_travel_advisor', call_id='call_pfsYzFfKVXUhzAzlYohQJSt0', is_error=False)]
---------- HandoffMessage (travel_concierge) ----------
전문 상담사에게 연결합니다


TaskResult(messages=[TextMessage(id='19d9b7d9-9855-4c01-875a-415d34ebd7f0', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 21, 6, 926942, tzinfo=datetime.timezone.utc), content='유럽 비자 신청 절차와 항공료 할인받는 방법, 여행자보험 추천까지 상세히 알려주세요', type='TextMessage'), ToolCallRequestEvent(id='28f82a2c-202c-455d-83a1-ab31b44a3add', source='travel_concierge', models_usage=RequestUsage(prompt_tokens=529, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 21, 7, 501026, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_pfsYzFfKVXUhzAzlYohQJSt0', arguments='{}', name='transfer_to_travel_advisor')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='3a4d3b60-c4fc-4921-be0e-d2c7be68c678', source='travel_concierge', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 21, 7, 502338, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='전문 상담사에게 연결합니다', name='transfer_to_travel_

## 🎓 실습 과제 및 확장 아이디어

### 📚 이번 실습에서 배운 내용:
1. **실시간 API 연동**: Google Places API와 OpenWeatherMap API 활용법
2. **Fallback 시스템**: API 키 없이도 동작하는 안전장치 구현
3. **단일 에이전트 팀**: 실제 API를 활용하는 전문 에이전트
4. **반성 팀**: Primary-Critic 협업으로 품질 향상
5. **핸드오프 팀**: 적절한 역할 분담과 에스컬레이션
6. **비동기 처리**: aiohttp를 활용한 효율적인 API 호출

### 🚀 추가 도전 과제:
1. **더 많은 API 연동**: 
   - 실시간 환율 API (ExchangeRate-API)
   - 항공료 검색 API (Amadeus, Skyscanner)
   - 호텔 가격 비교 API (Booking.com, Hotels.com)

2. **고급 에이전트 기능**:
   - 다국어 번역 API 연동
   - 이미지 분석 API (관광지 사진 분석)
   - 추천 시스템 (사용자 취향 기반)

3. **프로덕션 배포**:
   - FastAPI 웹 서버 구축
   - 데이터베이스 연결 (PostgreSQL, MongoDB)
   - Docker 컨테이너화

### 💡 핵심 포인트:
- **명확한 역할 분담**: 각 에이전트의 전문성 정의
- **적절한 종료 조건**: 무한 루프 방지와 효율성
- **사용자 경험 고려**: 언제 전문가에게 연결할지 판단
- **한국어 자연어 처리**: 문화적 맥락을 고려한 대화

---

## 🔥 보너스 실습: 실시간 API 연동 통합 여행 계획 시스템

위에서 만든 **실시간 API 연동 팀들**을 순서대로 실행하여 완전한 여행 계획을 세워보세요!

```python
# 실시간 API 연동 워크플로우:
# 1단계: 실시간 여행지 + 날씨 정보 → 2단계: 예산 계획 → 3단계: 전문 상담

async def complete_travel_planning_with_apis(destination: str, duration: str, style: str):
    print("🌐 1단계: 실시간 여행지 + 날씨 정보")
    # travel_expert_team 실행 (Google Places + OpenWeatherMap API)
    
    print("💰 2단계: 예산 계획") 
    # budget_management_team 실행 (예산 계산 + 검토)
    
    print("🛎️ 3단계: 전문 상담")
    # concierge_team 실행 (복잡한 요청 시 핸드오프)
```

### 🌟 API 키 설정 방법:
```bash
# .env 파일에 추가
GOOGLE_PLACES_API_KEY=your_google_api_key_here
OPENWEATHER_API_KEY=your_openweather_api_key_here
```

**이제 여러분만의 실시간 API 연동 AI 팀을 만들어보세요!** 🚀🌐